<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>External Programs</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>

## Subprocesses

*Note:* You need to have the `ext_sample_app` package (in
`Examples/ExternalSampleApplication`) installed to run the following examples.

`subprocess.run()` is the preferred way of running external applications.

In [ ]:
from subprocess import TimeoutExpired, run

In [ ]:
# This may not work if `python` is not in your path...
run(["python", "--version"])

With `shutil.which()` you can determine the full path of a program.

In [ ]:
import shutil

shutil.which("python")

In [ ]:
cp = run([shutil.which("python"), "--version"])

In [ ]:
def print_completed_process(cp):
    print("return code:", cp.returncode)
    print("captured stdout:", repr(cp.stdout))
    print("captured stderr:", repr(cp.stderr))

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run([shutil.which("python"), "--version"], capture_output=True, text=True)

In [ ]:
print_completed_process(cp)

With `sys.executable` you can find out the path of the currently active Python interpreter. This is the preferred way to start a Python process.

In [ ]:
import sys

cp = run([sys.executable, "--version"], capture_output=True, text=True)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run([sys.executable, "-m", "ext_sample_app"], capture_output=True, text=True)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "--help"], capture_output=True, text=True
)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "say-hi"], capture_output=True, text=True
)

In [ ]:
print_completed_process(cp)

In [ ]:
cp = run(
    [sys.executable, "-m", "ext_sample_app", "error"], capture_output=True, text=True
)

In [ ]:
print_completed_process(cp)

In [ ]:
# THIS DOES NOT WORK!
# cp = run(
#     [sys.executable, "-m", "ext_sample_app", "interact"], capture_output=True, text=True
# )

## Popen: Concurrent execution of programs

If you can't wait for the launched program to finish, you have to use the `subprocess.Popen` class:

In [ ]:
from subprocess import Popen, PIPE
import sys

In [ ]:
proc = Popen(
    [sys.executable, "-m", "ext_sample_app", "interact"],
    stdin=PIPE,
    stderr=PIPE,
    stdout=PIPE,
    encoding="utf-8",
    universal_newlines=True,
    bufsize=0,
)

In [ ]:
type(proc)

`proc.communicate()` sends a message to `proc`, closes the input and output streams and ends the process.

In [ ]:
proc.communicate("work")

With `proc.poll()` you can determine whether the process has already ended and what its return value was. If the result is `None`, the process is still active. `proc.wait()` waits a certain amount of time and returns the status of the process. If the process hasn't finished in the allotted time, a `TimeoutExpired` exception is thrown.

In [ ]:
proc.poll()

In [ ]:
def run_and_communicate(command):
    proc = Popen(
        [sys.executable, "-m", "ext_sample_app", "interact"],
        stdin=PIPE,
        stderr=PIPE,
        stdout=PIPE,
        encoding="utf-8",
        universal_newlines=True,
        bufsize=0,
    )
    result = proc.communicate(command)
    try:
        wait_result = proc.wait(5)
    except TimeoutExpired:
        print("Process did not terminate!")
        proc.terminate()
        wait_result = proc.wait(5)
    return result, wait_result

In [ ]:
run_and_communicate("work")

In [ ]:
run_and_communicate("exit")

In [ ]:
run_and_communicate("error")

## Communication with sockets

The following example shows how to start a process and then communicate with it using sockets.

In [ ]:
from subprocess import Popen, PIPE
import sys

HOST = "localhost"
PORT = 12345

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM
import sys


def send_message(msg: str):
    with socket(AF_INET, SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(bytes(msg + "\n", "utf-8"))
        return str(sock.recv(1024), "utf-8")

In [ ]:
proc = Popen(
    [
        sys.executable,
        "-m",
        "ext_sample_app",
        "serve",
        "--host",
        HOST,
        "--port",
        str(PORT),
    ],
    stdin=PIPE,
    stderr=PIPE,
    stdout=PIPE,
    encoding="utf-8",
    universal_newlines=True,
    bufsize=0,
)

In [ ]:
proc.poll()

In [ ]:
send_message("Hello, world!")

In [ ]:
send_message("Are you running?")

In [ ]:
proc.poll()

In [ ]:
proc.terminate()

In [ ]:
proc.poll()

In [ ]:
try:
    send_message("Are you still running?")
except ConnectionRefusedError as err:
    print("Could not connect to server.")
    print(err)

In [ ]:
proc.poll()